# ReAct Agents

Now we are going to code a ReAct agent. (see also: https://langchain-ai.github.io/langgraph/how-tos/react-agent-from-scratch/?h=react).
These are a widely used type of agent. 
ReAct stands for REasoning and ACT agent. 
Basically this agent has access to tools; he loops through tool nodes, performing tool calls, until he decides there is no need for more tool calling. 
When that happens, he stops its flows. 
See the image below:

<div style="text-align: center;">
  <img src="../images/react.png" alt="Basic Example of a graph in LangGraph" style="width: 70%;">
</div>


So we will: 
    - Learn how to create **tools** in LangGraph
    - How to create a ReAct Graph
    - Work with ToolMessages

The code is inside `0.8.10_ReAct_agent.py`.

We will use this notebook for further notes and insights. 

## Imports:

### `Annotated`


The `Annotated` type lets you attach **metadata** to a type hint without affecting the actual runtime behavior.

In [ ]:
from typing import Annotated #<- provides additional context without affecting the type itself

#example: 

email = Annotated[str, "This has to be a valid email format!"]  # we add metadata to our variable

print(email.__metadata__)   # allows us to inspect metadata

### `Sequence`

The `Sequence` type represents any **ordered, iterable collection** that supports indexing and `len()` — like lists, tuples, or strings.

```python
from typing import Sequence

def total_length(items: Sequence[str]) -> int:
    return sum(len(item) for item in items)
```
- Accepts both `list` and `tuple` (or any sequence-like object).
- Ensures items are indexable (`items[0]`), iterable, and have a length (`len(items)`).
Read-only: does not guarantee `.append()` or mutability.

### 🔁 `add_messages` from `langgraph.graph.message`

`add_messages` is a **reducer function** used in LangGraph. A reducer controls **how updates from nodes are merged into the existing state** during graph execution.

It is especially useful when working with message histories in an agent.

---

### Without a reducer (default overwrite)

```python
state = {"messages": ["Hi!"]}
update = {"messages": ["Nice to meet you!"]}

# result without a reducer:
new_state = {"messages": ["Nice to meet you!"]}
```

### With `add_messages` reducer (merge behavior)

```python
from langgraph.graph.message import add_messages

state = {"messages": ["Hi!"]}
update = {"messages": ["Nice to meet you!"]}

# result with add_messages:
new_state = {"messages": ["Hi!", "Nice to meet you!"]}
```
The `add_messages` reducer preserves the previous conversation, appending new messages to the existing list.


### 💬 `BaseMessage` in LangChain

`BaseMessage` is the **abstract base class** for all messages exchanged in a LangChain conversation. It's part of the `langchain_core.messages` module and defines the common interface for:

* Human messages (`HumanMessage`)
* AI messages (`AIMessage`)
* System messages (`SystemMessage`)
* Function or tool messages (e.g., `FunctionMessage`, `ToolMessage`)

---

### 🧱 Purpose

`BaseMessage` ensures that all message types:

* Store the actual **content** (usually a string)
* Track the **role** (e.g. `"human"`, `"ai"`, `"system"`, etc.)
* Optionally include **metadata** and **tool call information**

This uniform structure allows messages to be:

* Easily parsed by LLMs
* Logged and traced across a graph
* Passed as part of an agent's state

---

### 🧾 Example

```python
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content="Hello!"),
    AIMessage(content="Hi there! How can I help you today?")
]

for msg in messages:
    print(f"{msg.type}: {msg.content}")
```

---

### 🧠 Common Subclasses

| Class             | Role         | Use case                            |
| ----------------- | ------------ | ----------------------------------- |
| `HumanMessage`    | `"human"`    | User input                          |
| `AIMessage`       | `"ai"`       | LLM-generated reply                 |
| `SystemMessage`   | `"system"`   | Instructions or context for the LLM |
| `ToolMessage`     | `"tool"`     | Output from a tool call             |
| `FunctionMessage` | `"function"` | Function-calling responses          |

---

### 📌 Notes

* You should never instantiate `BaseMessage` directly. Always use its subclasses.
* All messages are typically stored in a `messages` list passed to agents or chains.
* Useful for creating conversation history in memory or routing logic.

Let me know if you want a section about serializing or customizing messages!


## Comments

#### 📌 `Annotated[Sequence[BaseMessage], add_messages]` explained

This type annotation is used in LangGraph to define:
- A list of LangChain messages (`HumanMessage`, `AIMessage`, etc.)
- With `add_messages` as a reducer function to **append** new messages to the existing list during state updates.

It allows automatic merging of conversation history as the agent runs.


> **Note:** Although `Annotated` is generally used to add metadata like descriptions or constraints, in LangGraph it is also used to attach a function — such as `add_messages`. This function acts as a reducer, meaning it defines how new data should be merged with the existing state. You're not calling the function directly; you're tagging the type with special behavior that LangGraph will use during execution to manage state updates.


#### 📌 `@tool` Decorator and `bind_tools()`

When we define tools we need to explicitly use the tool decorator, in order for our language model to know that it can use that function.

After defining the tools we give the model access to them by "binding" them with `bind_tools()`.